In [208]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from keras.layers import Activation, Dense, Input
from keras.layers import Conv2D, Flatten
from keras.layers import Reshape, Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import BatchNormalization
from keras import optimizers
from keras.models import Model
from keras.datasets import mnist

In [209]:
import numpy as np
import math
import os
import matplotlib.pyplot as plt

In [210]:
def NN_discriminator(inputs,kernel_size):
    #kernel_size: length of CNN window
    #input: image
    nfilter_layers=[4,8,16,32]*2
    x=inputs
    for nfilter in nfilter_layers:
        stride=2
        if nfilter==nfilter_layers[-1]:
            stride=1
        x=LeakyReLU(alpha=0.2)(x)
        x=Conv2D(filters=nfilter,kernel_size=kernel_size,strides=stride,padding='same')(x)
    x=Flatten()(x)
    x=Dense(1)(x)
    x=Activation('sigmoid')(x)
    discriminator=Model(inputs,x,name='Discriminator')
    return discriminator

In [211]:
def NN_generator(inputs,size_imag,kernel_size):
    nfilter_layers=[64,32,16,1]
    imag_resize=size_imag//4
    x=Dense(imag_resize*imag_resize*nfilter_layers[0])(inputs)
    x=Reshape((imag_resize,imag_resize,nfilter_layers[0]))(x)
    
    for nfilter in nfilter_layers:
        stride=2
        if nfilter>nfilter_layers[-2]:
            stride=1
        x=BatchNormalization()(x)
        x=Activation('relu')(x)
        x=Conv2DTranspose(filters=nfilter,kernel_size=kernel_size,strides=stride,padding='same')(x)
    x=Activation('sigmoid')(x)
    generator=Model(inputs,x,name='Generator')
    return generator

In [ ]:
a1=5
log="my test: a1= %d" %(a1)
print(log)

my test: a1= 5


In [ ]:
def plot_imag(generator,noise_input,show=False,step=0,model_name='gan'):
    """
    generator(model)
    noise_input(ndarray): array of latent vector
    """
    os.makedirs(model_name,exist_ok=True)
    filename=os.path.join(model_name,"%3d.png" %step)
    imags=generator.predict(noise_input)
    plt.figure(figsize=(10,10))
    n_imag=imags.shape[0]
    size_imag=imags.shape[1]
    cols=4
    rows=int(noise_input.shape[0]/4)
    for i in range(n_imag):
        plt.subplot(rows,cols,i+1)
        imag=np.reshape(imags[i],[size_imag,size_imag])
        plt.imshow(imag,cmap='gray')
        plt.axis('off')
    plt.savefig(filename)
    if show:
        plt.show()
    else:
        plt.close('all')
    

In [ ]:
def train_GAN(models,x_train,param):
    generator,discrim,gan=models
    batch_size, dim_latent, train_steps = param
    save_interval=1000
    noise=np.random.uniform(-1.0,1.0,size=[16,dim_latent])
    train_size=x_train.shape[0]
    for i in range(train_steps):
        id_rand=np.random.randint(0,train_size,size=batch_size)
        imag_real=x_train[id_rand]
        noise_input=np.random.uniform(-1.0,1.0,size=[batch_size,dim_latent])
        imag_fake=generator.predict(noise_input)
        x=np.concatenate((imag_real,imag_fake))
        y=np.ones([2*batch_size,1])
        y[batch_size:,:]=0.0
        loss,acc=discrim.train_on_batch(x,y)
        log="%d:discrim loss: %f, acc: %f" %(i,loss,acc)
        
        noise=np.random.uniform(-1.0,1.0,size=[batch_size,dim_latent])
        y=np.ones([batch_size,1])
        loss,acc=gan.train_on_batch(noise,y)
        log="%s; gan loss: %f, acc: %f" %(log,loss,acc)
        print(log)
        if(i+1)% save_interval==0:
            if (i+1)==train_steps:
                show_flag=True
            else:
                show_flag=False    
            plot_imag(generator,noise_input=noise_input,show=show_flag,step=(i+1),model_name='GAN')      
    generator.save("gan_mnist.h5")  
        

In [ ]:
def build_GAN():
    (x_train,_),(_,_)=mnist.load_data()
    size_imag=x_train.shape[1]
    x_train=np.reshape(x_train,[-1,size_imag,size_imag,1])
    x_train=x_train.astype('float32')/255
    
    dim_latent=50
    batch_size=32
    train_steps=50000
    lr=1e-3
    
    #build discriminator model
    input_shape=(size_imag,size_imag,1)
    input_discrim=Input(shape=input_shape,name='input_discrim')
    kernel_size_discrim=3
    discrim=NN_discriminator(input_discrim,kernel_size_discrim)
    optimizer=optimizers.Adam(lr=lr)
    discrim.compile(loss='binary_crossentropy',optimizer=optimizer,metrics=['accuracy'])
    discrim.summary()
    
    #build generator model
    kernel_size_gen=3
    input_shape=(dim_latent,)
    input_gen=Input(shape=input_shape,name='latent_input')
    generator=NN_generator(input_gen,size_imag,kernel_size_gen)
    generator.compile(loss='binary_crossentropy',optimizer=optimizer,metrics=['accuracy'])
    generator.summary()
    
    gan=Model(input_gen,discrim(generator(input_gen)),name='gan')
    gan.compile(loss='binary_crossentropy',optimizer=optimizer,metrics=['accuracy'])
    gan.summary()
    
    models=(generator,discrim,gan)
    param=(batch_size,dim_latent,train_steps)
    train_GAN(models,x_train,param)

In [ ]:
def test_generator(generator):
    noise_input=np.random.uniform(-1.0,1.0,size=[16,20])
    plot_imag(generator,noise_input,show=True,model_name='test_generator')

In [ ]:
build_GAN()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_discrim (InputLayer)   (None, 28, 28, 1)         0         
_________________________________________________________________
leaky_re_lu_85 (LeakyReLU)   (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_85 (Conv2D)           (None, 14, 14, 4)         40        
_________________________________________________________________
leaky_re_lu_86 (LeakyReLU)   (None, 14, 14, 4)         0         
_________________________________________________________________
conv2d_86 (Conv2D)           (None, 7, 7, 8)           296       
_________________________________________________________________
leaky_re_lu_87 (LeakyReLU)   (None, 7, 7, 8)           0         
_________________________________________________________________
conv2d_87 (Conv2D)           (None, 4, 4, 16)          1168      
__________

27:discrim loss: 0.708545, acc: 0.500000; gan loss: 0.253700, acc: 1.000000
28:discrim loss: 0.709766, acc: 0.500000; gan loss: 0.226231, acc: 1.000000
29:discrim loss: 0.710844, acc: 0.500000; gan loss: 0.202287, acc: 1.000000
30:discrim loss: 0.722034, acc: 0.500000; gan loss: 0.181655, acc: 1.000000
31:discrim loss: 0.721104, acc: 0.500000; gan loss: 0.165913, acc: 1.000000
32:discrim loss: 0.730906, acc: 0.500000; gan loss: 0.152353, acc: 1.000000
33:discrim loss: 0.727724, acc: 0.500000; gan loss: 0.142199, acc: 1.000000
34:discrim loss: 0.735064, acc: 0.500000; gan loss: 0.134353, acc: 1.000000
35:discrim loss: 0.738029, acc: 0.500000; gan loss: 0.128896, acc: 1.000000
36:discrim loss: 0.741782, acc: 0.500000; gan loss: 0.125203, acc: 1.000000
37:discrim loss: 0.737536, acc: 0.500000; gan loss: 0.123014, acc: 1.000000
38:discrim loss: 0.737327, acc: 0.500000; gan loss: 0.122213, acc: 1.000000
39:discrim loss: 0.738598, acc: 0.500000; gan loss: 0.122294, acc: 1.000000
40:discrim l

136:discrim loss: 0.000003, acc: 1.000000; gan loss: 0.000000, acc: 1.000000
137:discrim loss: 0.000008, acc: 1.000000; gan loss: 0.000000, acc: 1.000000
138:discrim loss: 0.000009, acc: 1.000000; gan loss: 0.000000, acc: 1.000000
139:discrim loss: 0.000003, acc: 1.000000; gan loss: 0.000000, acc: 1.000000
140:discrim loss: 0.000003, acc: 1.000000; gan loss: 0.000000, acc: 1.000000
141:discrim loss: 0.000006, acc: 1.000000; gan loss: 0.000000, acc: 1.000000
142:discrim loss: 0.000005, acc: 1.000000; gan loss: 0.000000, acc: 1.000000
143:discrim loss: 0.000051, acc: 1.000000; gan loss: 0.000000, acc: 1.000000
144:discrim loss: 0.000008, acc: 1.000000; gan loss: 0.000000, acc: 1.000000
145:discrim loss: 0.000012, acc: 1.000000; gan loss: 0.000000, acc: 1.000000
146:discrim loss: 0.000026, acc: 1.000000; gan loss: 0.000000, acc: 1.000000
147:discrim loss: 0.000014, acc: 1.000000; gan loss: 0.000000, acc: 1.000000
148:discrim loss: 0.000003, acc: 1.000000; gan loss: 0.000000, acc: 1.000000

244:discrim loss: 0.000537, acc: 1.000000; gan loss: 0.000000, acc: 1.000000
245:discrim loss: 0.000510, acc: 1.000000; gan loss: 0.000000, acc: 1.000000
246:discrim loss: 0.000745, acc: 1.000000; gan loss: 0.000000, acc: 1.000000
247:discrim loss: 0.000618, acc: 1.000000; gan loss: 0.000000, acc: 1.000000
248:discrim loss: 0.000257, acc: 1.000000; gan loss: 0.000000, acc: 1.000000
249:discrim loss: 0.000873, acc: 1.000000; gan loss: 0.000000, acc: 1.000000
250:discrim loss: 0.000367, acc: 1.000000; gan loss: 0.000000, acc: 1.000000
251:discrim loss: 0.002589, acc: 1.000000; gan loss: 0.000000, acc: 1.000000
252:discrim loss: 0.000244, acc: 1.000000; gan loss: 0.000001, acc: 1.000000
253:discrim loss: 0.000228, acc: 1.000000; gan loss: 0.000001, acc: 1.000000
254:discrim loss: 0.000342, acc: 1.000000; gan loss: 0.000001, acc: 1.000000
255:discrim loss: 0.000184, acc: 1.000000; gan loss: 0.000001, acc: 1.000000
256:discrim loss: 0.000231, acc: 1.000000; gan loss: 0.000001, acc: 1.000000

351:discrim loss: 0.679902, acc: 0.484375; gan loss: 0.520409, acc: 1.000000
352:discrim loss: 0.674950, acc: 0.468750; gan loss: 0.545151, acc: 0.968750
353:discrim loss: 0.641746, acc: 0.468750; gan loss: 0.562360, acc: 0.968750
354:discrim loss: 0.680748, acc: 0.390625; gan loss: 0.581468, acc: 0.968750
355:discrim loss: 0.627816, acc: 0.500000; gan loss: 0.609182, acc: 0.843750
356:discrim loss: 0.629092, acc: 0.484375; gan loss: 0.625423, acc: 0.812500
357:discrim loss: 0.642687, acc: 0.421875; gan loss: 0.638475, acc: 0.750000
358:discrim loss: 0.611218, acc: 0.406250; gan loss: 0.657549, acc: 0.718750
359:discrim loss: 0.616268, acc: 0.453125; gan loss: 0.679784, acc: 0.468750
360:discrim loss: 0.615984, acc: 0.453125; gan loss: 0.686415, acc: 0.562500
361:discrim loss: 0.590770, acc: 0.468750; gan loss: 0.694260, acc: 0.531250
362:discrim loss: 0.561389, acc: 0.500000; gan loss: 0.706524, acc: 0.343750
363:discrim loss: 0.574623, acc: 0.468750; gan loss: 0.717786, acc: 0.281250

...

4443:discrim loss: 0.803611, acc: 0.500000; gan loss: 0.430770, acc: 1.000000
4444:discrim loss: 0.804212, acc: 0.500000; gan loss: 0.430275, acc: 1.000000
4445:discrim loss: 0.803635, acc: 0.500000; gan loss: 0.428502, acc: 1.000000
4446:discrim loss: 0.805109, acc: 0.500000; gan loss: 0.426617, acc: 1.000000
4447:discrim loss: 0.806260, acc: 0.500000; gan loss: 0.425239, acc: 1.000000
4448:discrim loss: 0.803638, acc: 0.500000; gan loss: 0.424869, acc: 1.000000
4449:discrim loss: 0.803735, acc: 0.500000; gan loss: 0.424481, acc: 1.000000
4450:discrim loss: 0.804486, acc: 0.500000; gan loss: 0.424539, acc: 1.000000
4451:discrim loss: 0.803995, acc: 0.500000; gan loss: 0.423861, acc: 1.000000
4452:discrim loss: 0.804213, acc: 0.500000; gan loss: 0.426211, acc: 1.000000
4453:discrim loss: 0.802616, acc: 0.500000; gan loss: 0.425643, acc: 1.000000
4454:discrim loss: 0.800651, acc: 0.500000; gan loss: 0.426461, acc: 1.000000
4455:discrim loss: 0.800741, acc: 0.500000; gan loss: 0.426700, 

4550:discrim loss: 0.672812, acc: 0.500000; gan loss: 0.554588, acc: 1.000000
4551:discrim loss: 0.672656, acc: 0.500000; gan loss: 0.554483, acc: 1.000000
4552:discrim loss: 0.659335, acc: 0.500000; gan loss: 0.541687, acc: 1.000000
4553:discrim loss: 0.681014, acc: 0.500000; gan loss: 0.543634, acc: 1.000000
4554:discrim loss: 0.688038, acc: 0.500000; gan loss: 0.559720, acc: 1.000000
4555:discrim loss: 0.665754, acc: 0.500000; gan loss: 0.568973, acc: 1.000000
4556:discrim loss: 0.663921, acc: 0.500000; gan loss: 0.568064, acc: 1.000000
4557:discrim loss: 0.661901, acc: 0.500000; gan loss: 0.551459, acc: 1.000000
4558:discrim loss: 0.670632, acc: 0.500000; gan loss: 0.544263, acc: 1.000000
4559:discrim loss: 0.676861, acc: 0.500000; gan loss: 0.545629, acc: 1.000000
4560:discrim loss: 0.669652, acc: 0.500000; gan loss: 0.544520, acc: 1.000000
4561:discrim loss: 0.667996, acc: 0.500000; gan loss: 0.541777, acc: 1.000000
4562:discrim loss: 0.668668, acc: 0.500000; gan loss: 0.534903, 

4658:discrim loss: 0.507521, acc: 0.500000; gan loss: 0.606463, acc: 1.000000
4659:discrim loss: 0.497546, acc: 0.500000; gan loss: 0.605282, acc: 1.000000
4660:discrim loss: 0.501638, acc: 0.500000; gan loss: 0.606835, acc: 1.000000
4661:discrim loss: 0.498769, acc: 0.500000; gan loss: 0.607340, acc: 1.000000
4662:discrim loss: 0.503189, acc: 0.500000; gan loss: 0.607209, acc: 1.000000
4663:discrim loss: 0.498184, acc: 0.500000; gan loss: 0.606694, acc: 1.000000
4664:discrim loss: 0.511268, acc: 0.500000; gan loss: 0.600709, acc: 1.000000
4665:discrim loss: 0.496107, acc: 0.500000; gan loss: 0.594669, acc: 1.000000
4666:discrim loss: 0.504155, acc: 0.500000; gan loss: 0.590511, acc: 1.000000
4667:discrim loss: 0.499703, acc: 0.500000; gan loss: 0.582871, acc: 1.000000
4668:discrim loss: 0.503070, acc: 0.500000; gan loss: 0.577152, acc: 1.000000
4669:discrim loss: 0.507084, acc: 0.500000; gan loss: 0.571484, acc: 1.000000
4670:discrim loss: 0.512520, acc: 0.500000; gan loss: 0.567025, 

4765:discrim loss: 0.470123, acc: 0.500000; gan loss: 0.586210, acc: 1.000000
4766:discrim loss: 0.476980, acc: 0.500000; gan loss: 0.582873, acc: 1.000000
4767:discrim loss: 0.475842, acc: 0.500000; gan loss: 0.579944, acc: 1.000000
4768:discrim loss: 0.478660, acc: 0.500000; gan loss: 0.572413, acc: 1.000000
4769:discrim loss: 0.464154, acc: 0.500000; gan loss: 0.572178, acc: 1.000000
4770:discrim loss: 0.458541, acc: 0.500000; gan loss: 0.596400, acc: 1.000000
4771:discrim loss: 0.462981, acc: 0.500000; gan loss: 0.588321, acc: 1.000000
4772:discrim loss: 0.474918, acc: 0.500000; gan loss: 0.567498, acc: 1.000000
4773:discrim loss: 0.476444, acc: 0.500000; gan loss: 0.574250, acc: 1.000000
4774:discrim loss: 0.473931, acc: 0.500000; gan loss: 0.583717, acc: 1.000000
4775:discrim loss: 0.465478, acc: 0.500000; gan loss: 0.591026, acc: 1.000000
4776:discrim loss: 0.459055, acc: 0.500000; gan loss: 0.597304, acc: 1.000000
4777:discrim loss: 0.457387, acc: 0.500000; gan loss: 0.599430, 

4873:discrim loss: 0.444003, acc: 0.500000; gan loss: 0.605971, acc: 1.000000
4874:discrim loss: 0.445143, acc: 0.500000; gan loss: 0.604096, acc: 1.000000
4875:discrim loss: 0.445138, acc: 0.500000; gan loss: 0.602711, acc: 1.000000
4876:discrim loss: 0.446699, acc: 0.500000; gan loss: 0.600194, acc: 1.000000
4877:discrim loss: 0.447570, acc: 0.500000; gan loss: 0.597900, acc: 1.000000
4878:discrim loss: 0.449214, acc: 0.500000; gan loss: 0.595449, acc: 1.000000
4879:discrim loss: 0.451741, acc: 0.500000; gan loss: 0.593248, acc: 1.000000
4880:discrim loss: 0.452378, acc: 0.500000; gan loss: 0.590258, acc: 1.000000
4881:discrim loss: 0.455975, acc: 0.500000; gan loss: 0.588749, acc: 1.000000
4882:discrim loss: 0.458422, acc: 0.500000; gan loss: 0.585486, acc: 1.000000
4883:discrim loss: 0.457419, acc: 0.500000; gan loss: 0.583961, acc: 1.000000
4884:discrim loss: 0.459014, acc: 0.500000; gan loss: 0.581357, acc: 1.000000
4885:discrim loss: 0.462922, acc: 0.500000; gan loss: 0.580520, 